In [ ]:
# 导入工具包
import numpy as np
import argparse
import cv2


def order_points(pts):
    # 一共4个坐标点
    rect = np.zeros((4, 2), dtype = "float32")

    # 按顺序找到对应坐标0123分别是 左上，右上，右下，左下
    # 计算左上，右下
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]

    # 计算右上和左下
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]

    return rect


def four_point_transform(image, pts):
    # 获取输入坐标点
    rect = order_points(pts)  
    (tl, tr, br, bl) = rect

    # 计算输入的w和h值
    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    # 变换后对应坐标位置
    dst = np.array([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]], dtype = "float32")

    # 计算变换矩阵
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

    # 返回变换后结果
    return warped

def resize(image, width=None, height=None, inter=cv2.INTER_AREA):
dim = None
(h, w) = image.shape[:2]
if width is None and height is None:
    return image
if width is None:
    r = height / float(h)
    dim = (int(w * r), height)
else:
    r = width / float(w)
    dim = (width, int(h * r))
resized = cv2.resize(image, dim, interpolation=inter)
return resized

In [ ]:
# 读取输入
# image = cv2.imread(args["image"])
image = cv2.imread(r'C:\Users\ZHOU-JC\Desktop\scan.jpg')

#坐标也会相同变化
ratio = image.shape[0] / 500.0
orig = image.copy()


image = resize(orig, height = 500)

# 预处理
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(gray, 75, 200)

# 展示预处理结果
print("STEP 1: 边缘检测")
cv2.imshow("Image", image)
cv2.imshow("Edged", edged)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 轮廓检测
# cv2.RETR_LIST : 检测所有的轮廓
# cv2.CHAIN_APPROX_SIMPLE : 仅宝轮轮廓的拐点信息
cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[1]
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]

# 遍历轮廓
for c in cnts:
# 计算轮廓近似
peri = cv2.arcLength(c, True)
# C表示输入的点集
# epsilon表示从原始轮廓到近似轮廓的最大距离，它是一个准确度参数
# True表示封闭的
approx = cv2.approxPolyDP(c, 0.02 * peri, True)

# 4个点的时候就拿出来
if len(approx) == 4:
    screenCnt = approx
    break

# 展示结果
print("STEP 2: 获取轮廓")
cv2.drawContours(image, [screenCnt], -1, (0, 255, 0), 2)
cv2.imshow("Outline", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# 透视变换
warped = four_point_transform(orig, screenCnt.reshape(4, 2) * ratio)  # 关键一步

# 二值处理
warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
ref = cv2.threshold(warped, 100, 255, cv2.THRESH_BINARY)[1]
cv2.imwrite('scan.jpg', ref)
# 展示结果
print("STEP 3: 变换")
cv2.imshow("Original", resize(orig, height = 650))
cv2.imshow("Scanned", resize(ref, height = 650))
cv2.waitKey(0)

# util

In [2]:
# 导入工具包
from imutils import contours
import numpy as np
import argparse
import cv2

def cv_show(name,img):
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

- 将待识别图像 -> 灰度图像 -> 二值图像
- 通过轮廓检索函数 cv.findContours 找到待识别图像所有轮廓
- 模板图像 -> 灰度图像 -> 二值图像
- 通过轮廓检索函数 cv.findContours 找到模板图像中字母 A 的外轮廓
- 将第2步得到的轮廓逐一和第4步得到的轮廓 通过 cv.matchShapes 函数进行形状匹配。找到其中最小值，最小值对应的待识别图像中的轮廓即为匹配到的模板图像
- 标出在待识别图像中找到的模板图像

# 模板

In [195]:
template = cv2.imread('./template_resize.jpg')
template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
template = cv2.threshold(template, 200, 255, cv2.THRESH_BINARY_INV)[1]

cv_show('test', template)

In [201]:
template_y = template[172:260, 127:195]
cv_show('template_y', template_y)
cv2.imwrite('template_y.jpg', template_y)

True

In [202]:
template_z = template[174:261, 453:526]
cv_show('template_z', template_z)
cv2.imwrite('template_z.jpg', template_z)

True

In [203]:
template_jyfw = template[643:671, 71:226]
cv_show('template_jyfw', template_jyfw)
cv2.imwrite('template_jyfw.jpg', template_jyfw)

True

In [205]:
template_djjg = template[867:894, 337:458]
cv_show('template_djjg', template_djjg)
cv2.imwrite('template_djjg.jpg', template_djjg)

True

# 源图

In [124]:
image = cv2.imread('D:/Projects/sendi/OCR/Identify_Cards/data/2304_b9ae06cb41829198fc5b3a01036cc445.jpg')
cv_show('tmp', image)

In [125]:
# 原始图片 ->  resieze 高为1000 -> 灰色 -> 二值图像 + 提取黄色的二值图像

# resize
ratio = 1000 / image.shape[0]
image = cv2.resize(image, (0, 0), fx=ratio, fy=ratio, interpolation = cv2.INTER_LINEAR)
cv_show('tmp', image)

# 灰色
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv_show('gray', gray)

# 二值
black_white = cv2.threshold(gray, 100, 255, cv2.THRESH_BINARY_INV)[1]
cv_show('black_white', black_white)

# 提取黄色
low = 20
high = 30
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
lower_yellow = np.array([low, 50, 50])
upper_yellow = np.array([high, 255, 255])
mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
cv_show('test', mask)

# add
black_white =  cv2.add(black_white, mask)
cv_show('black_white', black_white)

In [126]:
_, contours, hierarchy =  cv2.findContours(black_white.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

edges =  cv2.drawContours(image.copy(), contours, -1, (0,255,0), 1)
cv_show('edges', edges)

In [127]:
# 初始化卷积核
rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (16, 15))
sqKernel   = cv2.getStructuringElement(cv2.MORPH_RECT, (16, 15))

# # 礼帽操作，突出更明亮的区域
# # 原图像-开运算图像 得到的是噪声图像
# tophat = cv2.morphologyEx(gray, cv2.MORPH_TOPHAT, rectKernel) 
# cv_show('tophat',tophat) 

gradX = cv2.Sobel(black_white, ddepth=cv2.CV_32F, dx=1, dy=0, ksize=-1)  # ksize = -1, 相当于用3*3的kernel
gradX = np.absolute(gradX)
(minVal, maxVal) = (np.min(gradX), np.max(gradX))
gradX = (255 * ((gradX - minVal) / (maxVal - minVal)))
gradX = gradX.astype("uint8")
cv_show('gradX', gradX) 

#通过闭操作（先膨胀，再腐蚀）将数字连在一起
gradX = cv2.morphologyEx(gradX, cv2.MORPH_CLOSE, rectKernel) 
cv_show('gradX',gradX)
# 二值操作
thresh = cv2.threshold(gradX, 100, 255, cv2.THRESH_BINARY)[1] 
cv_show('thresh',thresh)

# #再来一个闭操作
# thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, sqKernel) #再来一个闭操作
# cv_show('thresh',thresh)

## 计算轮廓

In [128]:
# 计算轮廓
thresh_, threshCnts, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)

cnts = threshCnts
cur_img = image.copy()
cv2.drawContours(cur_img, cnts,-1,(0,0,255),2) 
cv_show('img',cur_img)
locs = []

## 遍历轮廓

In [129]:
w0, h0 = image.shape[1], image.shape[0]

list_jyfw  = []
list_djjg   = []
list_y = []
list_z = []

# 遍历轮廓
for (i, c) in enumerate(cnts):
    # 计算矩形
    (x, y, w, h) = cv2.boundingRect(c)
    ar = w / float(h)
    
    # 选择合适的区域，根据实际任务来，这里的基本都是四个数字一组
    # 登记机关和经营范围
    if ar > 3 and ar < 12 and w > 20 and h > 10:
        if x > w0/2 and y > 0.7*h0:
            list_djjg.append((x, y, w, h))
        if x < w0*0.4 and y > h0*0.6 and y < h0*0.8 and x < 0.3*w0:
            list_jyfw.append((x, y, w, h))
    # 营 和 业
    if ar > 0.5 and ar < 2 and w > 30 and h > 30:
        if x < 0.4*w0 and x > 0.1*w0 and y < 0.3*h0:
            list_y.append((x, y, w, h))
        if x > 0.6*w0 and y < 0.3*h0:
            list_z.append((x, y, w, h))

## 匹配模板

In [130]:
temp = cv2.rectangle(image.copy(), (t1[0], t1[1]), (t1[0]+t1[2], t1[1]+t1[3]), (0, 0, 255))
cv_show('img',temp)

NameError: name 't1' is not defined

### 经营范围

In [131]:
template_jyfw = cv2.imread('./template_jyfw.jpg', cv2.COLOR_BAYER_BG2BGRA)
best_score = 0
result = ()

for (i, (gX, gY, gW, gH)) in enumerate(list_jyfw):
    # initialize the list of group digits
    # 根据坐标提取每一个组
    group = gray[gY - 5:gY + gH + 5, gX - 5:gX + gW + 5]
    cv_show('group',group)
    # 预处理
    group = cv2.threshold(group, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    cv_show('group',group)
    # resize
    roi = cv2.resize(group, (template_jyfw.shape[1], template_jyfw.shape[0]))
    cv_show('roi',roi)
    # 模板匹配
    score = cv2.matchTemplate(roi, template_jyfw, cv2.TM_CCOEFF)
    if score > best_score:
        best_score = score
        result = (gX, gY, gW, gH)

# 画出来
gX, gY, gW, gH = result
result_jyfw = result
temp = cv2.rectangle(image.copy(), (gX - 5, gY - 5), (gX + gW + 5, gY + gH + 5), (0, 0, 255), 1)
cv_show('temp', temp)

### 登记机关

In [132]:
template_djjg = cv2.imread('./template_djjg.jpg', cv2.COLOR_BAYER_BG2BGRA)
best_score = 0
result = ()

for (i, (gX, gY, gW, gH)) in enumerate(list_djjg):
    # initialize the list of group digits
    # 根据坐标提取每一个组
    group = gray[gY - 5:gY + gH + 5, gX - 5:gX + gW + 5]
    cv_show('group',group)
    # 预处理
    group = cv2.threshold(group, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    cv_show('group',group)
    # resize
    roi = cv2.resize(group, (template_djjg.shape[1], template_djjg.shape[0]))
    cv_show('roi',roi)
    # 模板匹配
    score = cv2.matchTemplate(roi, template_djjg, cv2.TM_CCOEFF)
    if score > best_score:
        best_score = score
        result = (gX, gY, gW, gH)

# 画出来
gX, gY, gW, gH = result
result_djjg = result
temp = cv2.rectangle(image.copy(), (gX - 5, gY - 5), (gX + gW + 5, gY + gH + 5), (0, 0, 255), 1)
cv_show('temp', temp)

### 营

In [133]:
template_y = cv2.imread('./template_y.jpg', cv2.COLOR_BAYER_BG2BGRA)
best_score = 0
result = ()

for (i, (gX, gY, gW, gH)) in enumerate(list_y):
    # initialize the list of group digits
    # 根据坐标提取每一个组
    group = gray[gY - 5:gY + gH + 5, gX - 5:gX + gW + 5]
    cv_show('group',group)
    # 预处理
    group = cv2.threshold(group, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    cv_show('group',group)
    # resize
    roi = cv2.resize(group, (template_y.shape[1], template_y.shape[0]))
    cv_show('roi',roi)
    # 模板匹配
    score = cv2.matchTemplate(roi, template_y, cv2.TM_CCOEFF)
    if score > best_score:
        best_score = score
        result = (gX, gY, gW, gH)

# 画出来
gX, gY, gW, gH = result
result_y = result
temp = cv2.rectangle(image.copy(), (gX - 5, gY - 5), (gX + gW + 5, gY + gH + 5), (0, 0, 255), 1)
cv_show('temp', temp)

### 照

In [134]:
template_z = cv2.imread('./template_z.jpg', cv2.COLOR_BAYER_BG2BGRA)
best_score = 0
result = ()

for (i, (gX, gY, gW, gH)) in enumerate(list_z):
    # initialize the list of group digits
    # 根据坐标提取每一个组
    group = gray[gY - 5:gY + gH + 5, gX - 5:gX + gW + 5]
    cv_show('group',group)
    # 预处理
    group = cv2.threshold(group, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    cv_show('group',group)
    # resize
    roi = cv2.resize(group, (template_z.shape[1], template_z.shape[0]))
    cv_show('roi',roi)
    # 模板匹配
    score = cv2.matchTemplate(roi, template_z, cv2.TM_CCOEFF)
    if score > best_score:
        best_score = score
        result = (gX, gY, gW, gH)

# 画出来
gX, gY, gW, gH = result
result_z = result
temp = cv2.rectangle(image.copy(), (gX - 5, gY - 5), (gX + gW + 5, gY + gH + 5), (0, 0, 255), 1)
cv_show('temp', temp)

# 投视变换

In [135]:
# 导入工具包
import numpy as np
import argparse
import cv2

# # 设置参数
# ap = argparse.ArgumentParser()
# ap.add_argument("-i", "--image", required = True,
# 	help = "Path to the image to be scanned")
# args = vars(ap.parse_args())


def four_point_transform(image):
    # 获取输入坐标点
    rect = np.array([result_y, result_z, result_jyfw, result_djjg])
    rect = rect[:, :2].astype('float32')
    
    # 变换后对应坐标位置
    dst = np.array([
        [127, 172],
        [453, 174],
        [71,  643],
        [337, 867]], dtype="float32")

    # 计算变换矩阵
    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (649, 1000))

    # 返回变换后结果
    return warped

In [136]:
img = four_point_transform(image)

In [137]:
cv_show('image', image)
cv_show('img', img)

# recognize

In [138]:
import requests
import json
import cv2
import base64

def cv2_to_base64(image):
    data = cv2.imencode(r'.jpg', image)[1]
    return base64.b64encode(data.tostring()).decode('utf8')

headers = {"Content-type": "application/json"}
url = "http://192.168.63.8:18889/predict/chinese_ocr_db_crnn_mobile"

# img_path = '/home/zhoujx/OCR/证件/营业执照/2956_2623a3d4c9e3c7c80eb43fe0f06ea275.jpg'
# img = cv2.imread(img_path)
img_base64 = cv2_to_base64(img)
# print(img_base64)
 
data = {'images': [img_base64]}
r = requests.post(url=url, headers=headers, data=json.dumps(data))
# print(r.json())
result = r.json()["results"]
print(result)
result_info = result[0]['data']
for re in result_info:
    print(re['text'])

D:\software\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  


[{'data': [{'confidence': 0.856296181678772, 'text': '营业热照', 'text_box_position': [[118, 169], [526, 167], [526, 259], [118, 261]]}, {'confidence': 0.9864295721054077, 'text': '统一社会信用代码', 'text_box_position': [[252, 291], [424, 293], [424, 316], [252, 314]]}, {'confidence': 0.9373663067817688, 'text': '92530924MA6KPE4B61', 'text_box_position': [[434, 296], [605, 297], [605, 314], [434, 313]]}, {'confidence': 0.9945915341377258, 'text': '经营者', 'text_box_position': [[69, 340], [176, 342], [176, 371], [69, 369]]}, {'confidence': 0.7125763297080994, 'text': '濮杏芳', 'text_box_position': [[190, 347], [251, 347], [251, 367], [190, 367]]}, {'confidence': 0.9809830784797668, 'text': '名', 'text_box_position': [[67, 393], [99, 393], [99, 422], [67, 422]]}, {'confidence': 0.5330525636672974, 'text': '称', 'text_box_position': [[141, 394], [179, 394], [179, 420], [141, 420]]}, {'confidence': 0.8535788655281067, 'text': '一镇康县南伞笼滋味串串香', 'text_box_position': [[175, 394], [397, 396], [397, 418], [175, 41

In [21]:
cv_show('test', img[435:486, 150:586])

In [283]:
cv_show('test', img)